# Must run individually based on work need to be done 

In [ ]:
#  CREATE INDEX ID_ACTOR for (n:actor) on (n.id);

In [34]:
from neo4j import GraphDatabase
from tqdm import tqdm
import pandas as pd

In [36]:
class Neo4j:
    
    def __init__(self,uri,user,password):
        self.driver = GraphDatabase.driver(uri,auth=(user,password))

    def query(self,query):
        result=None
        with self.driver.session() as session:
            result = list(session.run(query))
            session.close()
        return result
    
    def createyear(self,args):
        with self.driver.session() as session:
            session.run(f"CREATE (m:year $args)",args=args)
            session.close()
    def creategenre(self,args):
        with self.driver.session() as session:
            session.run(f"CREATE (m:genre $args)",args=args)
            session.close()
    def createmovie(self,args):
        with self.driver.session() as session:
            session.run(f"CREATE (m:movie $args)",args=args)
            session.close()
    def createwriter(self,args):
        with self.driver.session() as session:
            session.run(f"CREATE (m:writer $args)",args=args)
            session.close()
    def createdirector(self,args):
        with self.driver.session() as session:
            session.run(f"CREATE (m:director $args)",args=args)
            session.close()
    def createactor(self,args):
        with self.driver.session() as session:
            session.run(f"CREATE (m:actor $args)",args=args)
            session.close()
    def connectyear(self,year_name,movie_id):
        with self.driver.session() as session:
            query = f"""
                    MATCH(p:year)
                    WHERE p.name = $year_name
                    with p
                    MATCH (m:movie) 
                    WHERE m.id = $movie_id
                    WITH p,m
                    CREATE (m)-[r:`released-in`]->(p)
                    RETURN type(r)
                    """
            session.run(query,year_name=year_name,movie_id=movie_id)
            session.close()
    def connectgenre(self,genre_name,movie_id):
        with self.driver.session() as session:
            query = f"""
                    MATCH(p:genre)
                    WHERE p.name = $genre_name
                    with p
                    MATCH (m:movie) 
                    WHERE m.id = $movie_id
                    WITH p,m
                    CREATE (m)-[r:`genre-type`]->(p)
                    RETURN type(r)
                    """
            session.run(query,genre_name=genre_name,movie_id=movie_id)
            session.close()
    def connectdirector(self,director_id,movie_id):
        with self.driver.session() as session:
            query = f"""
                    MATCH(p:director)
                    WHERE p.id = $director_id
                    with p
                    MATCH (m:movie) 
                    WHERE m.id = $movie_id
                    WITH p,m
                    CREATE (p)-[r:directed]->(m)
                    RETURN type(r)
                    """
            session.run(query,director_id=director_id,movie_id=movie_id)
            session.close()
    def connectwriter(self,writer_id,movie_id):
        with self.driver.session() as session:
            query = f""" 
                    MATCH(p:writer)
                    WHERE p.id = $writer_id
                    with p
                    MATCH (m:movie) 
                    WHERE m.id = $movie_id
                    WITH p,m
                    CREATE (p)-[r:written]->(m)
                    RETURN type(r)
                    """
            session.run(query,writer_id=writer_id,movie_id=movie_id)
            session.close()
    def connectactor(self,actor_id,movie_id):
        with self.driver.session() as session:
            query = f"""
                    MATCH(p:actor)
                    WHERE p.id = $actor_id
                    with p
                    MATCH (m:movie) 
                    WHERE m.id = $movie_id
                    WITH p,m
                    CREATE (p)-[r:`acted-in`]->(m)
                    RETURN type(r)"""
            # query = f"""
            #         MATCH (p:actor),(m:movie)
            #         WHERE p.id = $actor_id AND m.id = $movie_id
            #         CREATE (p)-[r:`acted-in`]->(m)
            #         RETURN type(r)"""
            session.run(query,actor_id=actor_id,movie_id=movie_id)
            session.close()

    # def connect(self,user_id,movie_id,rating):
    #     with self.driver.session() as session:
    #         query = f"""
    #                 MATCH (p:Person),(m:Movie)
    #                 WHERE p.user_id = $user_id AND m.movie_id = $movie_id
    #                 CREATE (p)-[r:WATCHED]->(m) SET r.rating = $rating
    #                 RETURN type(r)"""
    #         session.run(query,user_id=user_id,movie_id=movie_id,rating=rating)
    #         session.close()
    
    def close(self):
        self.driver.close()

In [37]:
neo = Neo4j("bolt://localhost:7687","neo4j","1234")

# Genre nodes Create

In [23]:
Gnrs=set()
for filenum in range(1,19):
    df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Movies/results/result'+str(filenum)+'.csv')
    genres=df['genre']
    for g in range(0,len(genres)):
        split_genres=genres[g].split("|")
        for splitted_g in split_genres:
            Gnrs.add(splitted_g)
    # break
for each_G in Gnrs:
    if(each_G!="None"):
        neo.creategenre({"name":each_G})

# Year nodes Create

In [24]:
Yrs=set()
for filenum in range(1,19):
    df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Movies/results/result'+str(filenum)+'.csv')
    years=df['year']
    for y in range(0,len(years)):
        # split_years=years[y].split("|")
        # for splitted_y in split_years:
        Yrs.add(str(years[y]))
    # break
print(Yrs)
print(len(Yrs))
for each_Y in Yrs:
    if(str(each_Y)!="None"):
        neo.createyear({"name":str(each_Y)})

{'1987', '1959', '1916', '1979', '1961', '1968', '1955', '1939', '1947', '1937', '1954', '1906', '2020', '1931', '2005', '2006', '2003', '2007', '1946', '1926', '2011', '1935', '1980', '2014', '1965', '1999', '1932', '1938', '1948', '1977', '1973', '1989', '1915', '1944', '1964', '1911', '1942', '2010', '1929', '1978', '1972', '1984', '1996', '1925', '1969', '1998', '1962', '2002', '2009', '1949', '2021', '1930', '2013', '1970', '1921', '1985', '1919', '1986', '1990', '1967', '1914', '1917', '1994', '1988', '2018', '1983', '1981', '1924', '1940', '1952', '1971', '1963', '1928', '1953', '1927', '1951', '1975', '2017', '1912', '1960', '1974', '2008', '1950', '1958', '1920', '1913', '1945', '2000', '2019', '1934', '1936', '1997', '2004', '2012', '1923', '1933', '1894', '1966', '1941', '1957', '2001', '1922', '2015', '1976', '1991', 'None', '2016', '1992', '1995', '1956', '1918', '1943', '1993', '1982'}
114


# to get column names of movies

In [25]:
df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Movies/results/result1.csv')
columns=[]
for i in df.columns:
    columns.append(i)
columns.pop(0)
columns.remove("cast")
columns.remove("writers")
columns.remove("directedBy")
print(columns)

['id', 'year', 'Movie', 'genre', 'languages', 'rating', 'votings', 'duration', 'Countries', 'plot', 'small cover', 'fullsize cover']


In [26]:
for filenum in range(1,19):
    df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Movies/results/result'+str(filenum)+'.csv')
    # print(df['id'][0])
    for row in tqdm(df.iterrows()):
        args={}
        for i in columns:
            args[i]=row[1][i]
        neo.createmovie(args)
        # break

4999it [00:17, 278.75it/s]
5000it [00:13, 374.93it/s]
5000it [00:10, 498.42it/s]
5000it [00:10, 496.31it/s]
5000it [00:10, 459.96it/s]
5000it [00:10, 463.45it/s]
5000it [00:09, 527.32it/s]
5000it [00:11, 454.21it/s]
5000it [00:10, 455.14it/s]
5000it [00:11, 445.08it/s]
5000it [00:10, 456.80it/s]
5000it [00:12, 415.08it/s]
5000it [00:12, 411.23it/s]
5000it [00:12, 398.13it/s]
5000it [00:12, 408.14it/s]
5000it [00:12, 399.09it/s]
5000it [00:12, 390.29it/s]
856it [00:02, 410.55it/s]


# to get column names of writers

In [27]:
df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Writers/results/Writers1.csv')
columns=[]
for i in df.columns:
    columns.append(i)
columns.pop(0)
print(columns)

['id', 'name', 'Birth Date', 'Birth Notes', 'Biography', 'Trivia', 'Quotes', 'Headshot', 'Full-size Headshot']


In [28]:
for filenum in range(1,17):
    df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Writers/results/Writers'+str(filenum)+'.csv')
    # print(df['id'][0])
    for row in tqdm(df.iterrows()):
        args={}
        for i in columns:
            args[i]=row[1][i]
        neo.createwriter(args)

4999it [00:11, 422.71it/s]
5000it [00:10, 468.19it/s]
5000it [00:11, 451.22it/s]
5000it [00:11, 444.17it/s]
5000it [00:11, 441.42it/s]
5000it [00:10, 490.94it/s]
5000it [00:11, 452.31it/s]
5000it [00:11, 445.77it/s]
5000it [00:10, 477.62it/s]
5000it [00:11, 452.88it/s]
4999it [00:09, 515.82it/s]
5000it [00:10, 497.58it/s]
5000it [00:10, 467.27it/s]
5000it [00:11, 434.69it/s]
5000it [00:11, 436.61it/s]
3840it [00:08, 450.63it/s]


# to get columns for directors

In [29]:
df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Directors/results/Directors1.csv')
columns=[]
for i in df.columns:
    columns.append(i)
columns.pop(0)
print(columns)

['id', 'name', 'Birth Date', 'Birth Notes', 'Biography', 'Trivia', 'Quotes', 'Headshot', 'Full-size Headshot']


In [30]:
for filenum in range(1,9):
    df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Directors/results/Directors'+str(filenum)+'.csv')
    # print(df['id'][0])
    for row in tqdm(df.iterrows()):
        args={}
        for i in columns:
            args[i]=row[1][i]
        neo.createdirector(args)

4999it [00:11, 436.44it/s]
5000it [00:10, 468.16it/s]
5000it [00:10, 467.99it/s]
5000it [00:11, 434.82it/s]
5000it [00:11, 451.24it/s]
5000it [00:10, 498.26it/s]
4999it [00:10, 484.96it/s]
1640it [00:03, 468.17it/s]


# to get columns for cast

In [32]:
df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Cast/results/cast1.csv')
columns=[]
for i in df.columns:
    columns.append(i)
columns.pop(0)
print(columns)

['id', 'name', 'Birth Date', 'Birth Notes', 'Biography', 'Trivia', 'Quotes', 'Headshot', 'Full-size Headshot']


In [33]:
for filenum in range(1,219):
    df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Cast/results/cast'+str(filenum)+'.csv')
    # print(df['id'][0])
    for row in tqdm(df.iterrows()):
        args={}
        for i in columns:
            args[i]=row[1][i]
        neo.createactor(args)

5000it [00:10, 463.18it/s]
5000it [00:10, 458.00it/s]
5000it [00:10, 472.12it/s]
5000it [00:09, 548.06it/s]
5000it [00:09, 520.14it/s]
5000it [00:09, 508.49it/s]
5000it [00:09, 510.73it/s]
5000it [00:10, 495.54it/s]
5000it [00:11, 426.33it/s]
5000it [00:13, 365.76it/s]
5000it [00:11, 441.62it/s]
5000it [00:10, 455.11it/s]
5000it [00:11, 452.55it/s]
5000it [00:11, 437.57it/s]
5000it [00:11, 420.38it/s]
5000it [00:11, 423.30it/s]
5000it [00:12, 391.86it/s]
5000it [00:12, 411.85it/s]
5000it [00:10, 465.55it/s]
5000it [00:10, 474.88it/s]
5000it [00:10, 480.46it/s]
5000it [00:09, 501.00it/s]
5000it [00:10, 497.36it/s]
5000it [00:10, 486.00it/s]
5000it [00:10, 492.90it/s]
5000it [00:10, 457.24it/s]
5000it [00:11, 448.18it/s]
5000it [00:09, 502.01it/s]
5000it [00:10, 493.28it/s]
5000it [00:10, 499.11it/s]
5000it [00:10, 496.79it/s]
5000it [00:10, 456.64it/s]
5000it [00:09, 506.32it/s]
5000it [00:09, 503.47it/s]
5000it [00:09, 501.35it/s]
5000it [00:10, 493.70it/s]
5000it [00:10, 491.21it/s]
5

# Relations at Once

In [43]:
p=""
p=p.split("|")
s=set(p)
len(s)

61

In [40]:
for filenum in range(5,19):   
    df = pd.read_csv('C:/Users/grpnp/Desktop/Exercise1/DATA_COLLECTING/Movies/results/result'+str(filenum)+'.csv')
    movies_ids=df['id']
    year=df["year"]
    genres=df["genre"]
    directors_ids=df['directedBy']
    writers_ids=df['writers']
    cast_ids=df['cast']
    for i in tqdm(range(0,len(movies_ids))):
        movie_id=movies_ids[i]
        #year
        year_name=str(year[i])
        if(year_name!="None"):
            neo.connectyear(str(year[i]),movie_id)
        #genre    
        genre_set=set()
        for genre in genres[i].split("|"):
            if(genre!="None"):
                genre_set.add(genre)
        for u_genre in genre_set:
            neo.connectgenre(u_genre,movie_id)
        #director
        director_set=set()
        for director in directors_ids[i].split("|"):
            if(director!="None"):
                director_set.add(int(director))
        for u_director in director_set:
            neo.connectdirector(u_director,movie_id)
        # #writer
        writer_set=set()
        for writer in writers_ids[i].split("|"):
            if(writer!="None"):
                writer_set.add(int(writer))
        for u_writer in writer_set:
            neo.connectwriter(u_writer,movie_id)
        #cast
        cast_set=set()
        for cast in cast_ids[i].split("|"):
            if(cast!="None"):
                cast_set.add(int(cast))
        for u_cast in cast_set:
            neo.connectactor(u_cast,movie_id)


100%|██████████| 856/856 [00:49<00:00, 17.32it/s]


In [1]:
neo.close()

NameError: name 'neo' is not defined

# delete duplicate
MATCH (g:director) 
WITH g.id as id, collect(g) AS nodes 
WHERE size(nodes) >  1
FOREACH (g in tail(nodes) | DELETE g)

Movie: Home, Sweet Home